In [1]:
import json
import re

import pandas as pd


from collections import Counter
from pprint import pprint

In [2]:
directory_path = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE/PilotStudyToEvaluate_DATA_LABELS_2024-08-12_0908.csv'

df = pd.read_csv(directory_path)
df

,Study ID,Date subject signed consent,Date of Admission,Age at Admission,Sex,Ethnicity,Smoking History,Does the patient have any relevant medical history? (check all that apply) (choice=None),Does the patient have any relevant medical history? (check all that apply) (choice=Diabetes Mellitus),Does the patient have any relevant medical history? (check all that apply) (choice=Chronic Kidney Disease),...,Any adverse events?.5,"If yes, please specify.5",Is the subject compliant to medications?.5,List all the medications the subject is taking.5,Complete?.6,Date of discharge,Has patient completed study?,Date of study completion,Comments.1,Complete?.7
0,1,14/6/2022,13/6/2022,89,Male,Indian,Never smoked,Unchecked,Checked,Unchecked,...,NaN,NaN,NaN,NaN,Complete,15/6/2022,Yes,15/6/2022,NaN,Complete
1,2,16/6/2022,15/6/2022,68,Male,Chinese,Smoker,Unchecked,Checked,Checked,...,No,NaN,Yes,Aspirin 100mg OM Nifedipine LA 30mg BD Omepr...,Complete,22/6/2022,Yes,21/6/2022,NaN,Complete
2,3,20/6/2022,18/7/2022,81,Male,Chinese,Ex-smoker,Unchecked,Checked,Checked,...,NaN,NaN,NaN,NaN,Complete,23/6/2022,Yes,23/6/2022,NaN,Complete
3,4,18/6/2022,18/6/2022,55,Male,Malay,Ex-smoker,Unchecked,Checked,Unchecked,...,NaN,NaN,NaN,NaN,Complete,22/6/2022,Yes,22/6/2022,NaN,Complete
4,5,21/6/2022,20/6/2022,37,Female,Malay,Never smoked,Unchecked,Checked,Unchecked,...,No,NaN,Yes,ENGERIX B (Hepatitis B Vaccine) 20mcg/mL Injec...,Complete,26/6/2022,Yes,26/7/2022,NaN,Complete
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,66,20/12/2022,18/12/2022,63,Male,Indian,Smoker,Unchecked,Checked,Unchecked,...,NaN,NaN,NaN,NaN,Complete,29/12/2022,Yes,29/12/2022,NaN,Complete
66,67,20/12/2022,19/12/2022,55,Female,Malay,Never smoked,Checked,Unchecked,Unchecked,...,NaN,NaN,NaN,NaN,Complete,22/12/2022,Yes,22/12/2022,NaN,Complete
67,68,6/1/2023,5/1/2023,64,Male,Chinese,Ex-smoker,Unchecked,Unchecked,Unchecked,...,NaN,NaN,NaN,NaN,Complete,9/1/2023,Yes,9/1/2023,NaN,Complete
68,69,9/1/2023,7/1/2023,52,Male,Malay,Smoker,Unchecked,Unchecked,Unchecked,...,NaN,NaN,NaN,NaN,Complete,11/1/2023,Yes,11/1/2023,NaN,Complete


In [3]:
columns = list(df.columns)

list_pointers = []
list_pointers.append((0, 'start'))


counter_test = 0
counter_visit = 0
counter_discharge = 0

for i in range(len(columns)):
    column = columns[i]
    
    if ('Date and time of test' in column) or ('Date/time of test' in column):
        # if 'test' not in list_pointers[-1][1]:        
        list_pointers.append((i, f'test_{counter_test}'))
        counter_test += 1
        
    if ('Date of visit' in column):
        list_pointers.append((i, f'visit_{counter_visit}'))
        counter_visit += 1
        
    if ('Date of discharge' in column):
        list_pointers.append((i, 'discharge')) 
        counter_discharge += 1
        
list_pointers.append((i, 'end'))
pprint(list_pointers)

[(0, 'start'),
 (21, 'test_0'),
 (23, 'test_1'),
 (25, 'test_2'),
 (29, 'visit_0'),
 (54, 'test_3'),
 (61, 'test_4'),
 (68, 'test_5'),
 (76, 'visit_1'),
 (101, 'test_6'),
 (108, 'test_7'),
 (115, 'test_8'),
 (123, 'visit_2'),
 (148, 'test_9'),
 (155, 'test_10'),
 (162, 'test_11'),
 (170, 'visit_3'),
 (195, 'test_12'),
 (202, 'test_13'),
 (209, 'test_14'),
 (217, 'visit_4'),
 (242, 'test_15'),
 (249, 'test_16'),
 (256, 'test_17'),
 (264, 'visit_5'),
 (289, 'test_18'),
 (296, 'test_19'),
 (303, 'test_20'),
 (310, 'discharge'),
 (314, 'end')]


In [4]:
for i in range(len(list_pointers)-1):
    pointer_start = list_pointers[i][0]
    pointer_end = list_pointers[i+1][0]
    
    title = list_pointers[i][1]
    
    column_subset = columns[pointer_start:pointer_end]
    
    print('')
    print(title, '-', len(column_subset))
    pprint(column_subset)


start - 21
['Study ID',
 'Date subject signed consent',
 'Date of Admission',
 'Age at Admission',
 'Sex',
 'Ethnicity',
 'Smoking History',
 'Does the patient have any relevant medical history? (check all that apply) '
 '(choice=None)',
 'Does the patient have any relevant medical history? (check all that apply) '
 '(choice=Diabetes Mellitus)',
 'Does the patient have any relevant medical history? (check all that apply) '
 '(choice=Chronic Kidney Disease)',
 'Does the patient have any relevant medical history? (check all that apply) '
 '(choice=Hypertension)',
 'Does the patient have any relevant medical history? (check all that apply) '
 '(choice=Dyslipidemia)',
 'Does the patient have any relevant medical history? (check all that apply) '
 '(choice=Ischemic Heart Disease)',
 'Does the patient have any relevant medical history? (check all that apply) '
 '(choice=Atrial fibrillation)',
 'Does the patient have any relevant medical history? (check all that apply) '
 '(choice=Valvular H

# Reorganize Data

In [5]:
def remove_suffix_from_dict_keys(data):
    return {re.sub(r'\.\d+$', '', key).strip(): value for key, value in data.items()}


list_subject = df['Study ID'].tolist()

dict_subject = {}

for subject in list_subject:
    
    # print(subject)
    dict_subject[subject] = {
        'Visit': {}, 
        'Test': {},
        'Information': {}
        }
    
    df_subject = df[df['Study ID'] == subject].reset_index(drop=True)
    
    for i in range(len(list_pointers)-1):
        pointer_start = list_pointers[i][0]
        pointer_end = list_pointers[i+1][0]
        title = list_pointers[i][1]
        column_subset = columns[pointer_start:pointer_end]

        json_load = df_subject[column_subset].to_json(orient='records')
        json_load = json.loads(json_load)[0]
        
        json_load = remove_suffix_from_dict_keys(json_load)
    
        # def print_key_values(data, words):
        #     for key, value in data.items():
        #         if any(word in key for word in words):
        #             print(f"{key}: {value}")

        # # Example usage
        # specific_words = ["Date and time of test", 'Date/time of test']
        # print_key_values(json_load, specific_words)
        
        if 'Date/time of test' in json_load.keys():
            if json_load['Date/time of test'] == None:
                continue
            
        if 'Date and time of test' in json_load.keys():
            if json_load['Date and time of test'] == None:
                continue
        
        if 'Date of visit' in json_load.keys():
            if json_load['Date of visit'] == None:
                continue
    
        if 'test' in title:   
           dict_subject[subject]['Test'][title] = json_load
        elif 'visit' in title:
           dict_subject[subject]['Visit'][title] = json_load
        else:
           dict_subject[subject]['Information'][title] = json_load
    

# Get Subject Information

In [6]:
df_summary = []
for subject in list_subject:
    temp = dict_subject[subject]['Information'].copy()
    temp_start = temp['start'].copy()
    temp_discharge = temp['discharge'].copy()
    
    # Change key name
    # temp_discharge['Comments_Discharge'] = temp_discharge.pop('Comments')
    
    temp_start.update(temp_discharge)
    # print(temp_start)
    
    df_summary.append(temp_start)
    
df_summary = pd.DataFrame(df_summary)
df_summary.to_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE/subject_summary.csv', index=False)
# print(df_summary)

# Get Visits

In [7]:
df_summary = []
for subject in list_subject:
    temp = dict_subject[subject]['Visit'].copy()

    
    for key, value in temp.items():
        
        dict_info = {
            'Subject ID': subject,
            'Visit ID': key.replace('visit_', '')}
        
        dict_info.update(value)
        # print(dict_info)
        
        df_summary.append(dict_info)
    
df_summary = pd.DataFrame(df_summary)

# Preprocess
df_summary['Weight'] = df_summary['Weight'].apply(lambda x: str(x).replace('kg', '').strip() if x is not None else x)
df_summary['Weight'] = df_summary['Weight'].apply(lambda x: str(x).replace('..', '.').strip() if x is not None else x)

df_summary['SPO2'] = df_summary['SPO2'].apply(lambda x: str(x).replace('%', '').strip() if x is not None else x)

leg_columns_all = [
    'Left foot length (cm)', 
    'Left foot width (cm)', 
    'Left foot circumference at center (cm)', 
    'Left foot ankle circumference (cm)',
    'Right foot length (cm)',
    'Right foot width (cm)', 
    'Right foot circumference at center (cm)',
    'Right foot ankle circumference (cm)'
    ]

for col in leg_columns_all:
    df_summary[col] = df_summary[col].apply(lambda x: str(x).replace('..', '.').strip() if x is not None else x)


column = 'Right foot circumference at center (cm)'
df_summary[column] = df_summary[column].apply(lambda x: str(x).replace('/', '.').strip() if x is not None else x)

# Preprocess weight
column = 'How is the subject feeling today'
df_summary[column] = df_summary[column].apply(lambda x: str(x).lower() if x is not None else x)

df_summary = df_summary.fillna('')
df_summary.to_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE/subject_visit.csv', index=False)
# print(df_summary)
    
    

# Get Tests

In [8]:
df_summary = []
for subject in list_subject:
    temp = dict_subject[subject]['Test'].copy()

    
    for key, value in temp.items():
        
        # Update all date time to same column
        if 'Date/time of test' in value.keys():
            value['Date and time of test'] = value.pop('Date/time of test')
            
        # Get type of test
        test_type = ''
        if 'Troponin' in value.keys():
            test_type = 'Heart'
        elif 'Sodium' in value.keys():
            test_type = 'Kidney'
        elif 'Hemoglobin' in value.keys():
            test_type = 'Blood'
        elif 'NT-ProBNP' in value.keys():
            test_type = 'Clinical Trials'
    
        dict_info = {
            'Subject ID': subject,
            'Test ID': key.replace('test_', ''),
            'Test Type': test_type}
        
        dict_info.update(value)
        # print(dict_info)
        
        df_summary.append(dict_info)
    
df_summary = pd.DataFrame(df_summary)

df_summary = df_summary.fillna('')

df_summary['Date and time of test'] = pd.to_datetime(df_summary['Date and time of test'])
df_summary = df_summary.groupby('Subject ID').apply(lambda x: x.sort_values(by='Date and time of test')).reset_index(drop=True)

df_summary.to_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE/subject_test.csv', index=False)
# print(df_summary)
    
    

In [9]:
Counter(df_summary['Test Type'].tolist())

Counter({'Kidney': 202, 'Heart': 125, 'Blood': 88, 'Clinical Trials': 55})

In [10]:
len(df_summary)

470